<a href="https://colab.research.google.com/github/khalil-alexander/get_forecast/blob/main/FuturePositionISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <font color="red"> Pip install Astropy in google colab <font/>

In [183]:
!pip install astropy

## <font color="red"> Install Moudles in google colab <font/>

In [184]:
from astropy import coordinates as coord
from astropy import units as u
from astropy import time
from astropy.time import Time
from astropy.coordinates import Angle

In [185]:
import pandas as pd
from bs4 import BeautifulSoup
import requests as reqs
import xml.etree.ElementTree as ET
import io
import pprint
import json
import datetime

In [186]:
from datetime import datetime
import datetime as dt
from datetime import timezone

## <font color="red"> Converting coordinates to Latitude and Longitude ISS</font>

In [187]:
now = Time("2024-03-07 00:46:00.000", scale='utc')
xyz = [1155.746046202530, -6632.420367726780, 953.533229633281]
cartrep = coord.CartesianRepresentation(*xyz, unit=u.km)



gcrs = coord.GCRS(cartrep, obstime = now)
itrs = gcrs.transform_to(coord.ITRS(obstime = now))
loc = coord.EarthLocation(*itrs.cartesian.xyz)

print(loc.lat, loc.lon, loc.height)

8d07m57.90103915s 103d26m20.85144985s 421.8445702007271 km


## <font color="red"> Creating functions to clean up Latitude and Longitude ISS</font>

### <font color="green">Create an empty dateframe for functions

### <font color="green">Create two function that changes km to lat/lon


In [188]:
col_name = ['Date','x units(km)','y units(km)','z units(km)','x_dot(km/s)','y_dot(km/s)','z_dot(km/s)']
# Challenge to have the columns read under beautiful soup
xml_df = pd.DataFrame(columns=col_name)
xml_df

,Date,x units(km),y units(km),z units(km),x_dot(km/s),y_dot(km/s),z_dot(km/s)


In [189]:
def create_latlon_positions(date: str, xyz: list[float]):
    now = Time(date, scale='utc')
    cartrep = coord.CartesianRepresentation(*xyz, unit=u.km)

    gcrs = coord.GCRS(cartrep, obstime = now)
    itrs = gcrs.transform_to(coord.ITRS(obstime = now))
    loc = coord.EarthLocation(*itrs.cartesian.xyz)

    return Angle(loc.lat).deg, Angle(loc.lon).deg


In [190]:
def get_latlon(row: pd.Series):
    date = row['Date'].strftime('%Y-%m-%d %H:%M:%S')
    xyz = [row['x units(km)'], row['y units(km)'], row['z units(km)']]
    return create_latlon_positions(date, xyz)




## <font color="red"> read_xml_DateFrame
### <font color="green">A function that reads an xml file and returns the specfic position the iss is at during different time.

In [191]:
def read_xml_DataFrame(url):
    """
    Do web scraping from an XML file to obtain time the International Space
    Station will be at a specfic postion and the speed it is going at.

    Parameters
    ----------
    url : String
        The url of the data in XML format you want to extract from.

    Return
    ------
    df : Pandas DataFrame
        The DataFrame containing the time the International Space Station is at
        a specfic position and the speed it is traveling.
    """
    # Get the xml request
    xml = reqs.get(url)

    # Use BeautifulSoup to Organize the XML file
    soup = BeautifulSoup(xml.content,"lxml")

    # Finds all the instances 'statevector' appeared in the xml file
    state_tag = soup.find_all('statevector')

    col_name = ['Date','x units(km)',
                'y units(km)','z units(km)',
                'x_dot(km/s)',
                'y_dot(km/s)','z_dot(km/s)']
    df = pd.DataFrame(columns=col_name)

    #Loops thorugh all elements to add rows in the empty xml_df
    for element in state_tag:
        # Turns element into text and splits the '\n'
        element = element.text.split('\n')
        # Deletes First empty String
        element.pop(0)
        # Deletes Last empty Sting
        element.pop(-1)
        # Removes the 'Z' in the elements first index
        element[0] = element[0][:-1]

        # Appending list into DataFrame by adding 1 row to the DataFrame
        #df = xml_df.append(pd.DataFrame(element, columns=col_name), ignore_index=True)
        df.loc[len(df)] = element


    # Turns the "Date" collumn into a Datetime object
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%jT%H:%M:%S.%f')
    # Sets the "Date" collumn into a Datetime object
    #df.set_index("Date", inplace=True)

    # Returns the Dataframe
    return df

In [192]:
xml_url = "https://nasa-public-data.s3.amazonaws.com/iss-coords/current/ISS_OEM/ISS.OEM_J2K_EPH.xml"
%time df = read_xml_DataFrame(xml_url)

<ipython-input-191-ea97148a76b9>:21: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(xml.content,"lxml")


CPU times: user 12.7 s, sys: 84.5 ms, total: 12.8 s
Wall time: 19.8 s


In [250]:
df

,Date,lat,lon
0,2024-12-02 12:00:00,50.762893,24.239540
1,2024-12-02 12:04:00,51.429055,47.930295
2,2024-12-02 12:08:00,47.101542,69.774890
3,2024-12-02 12:12:00,39.058129,86.957239
4,2024-12-02 12:16:00,28.791850,100.005483
...,...,...,...
5397,2024-12-17 11:44:00,-51.361721,144.653539
5398,2024-12-17 11:48:00,-46.896149,166.347864
5399,2024-12-17 11:52:00,-38.765664,-176.623208
5400,2024-12-17 11:56:00,-28.444661,-163.673690


#### <font color="reeee">Adds latitude and longitude to the Data Frame

In [194]:
%time df[['lat','lon']] = df.apply(get_latlon, axis=1, result_type='expand')

CPU times: user 52.8 s, sys: 823 ms, total: 53.6 s
Wall time: 59.8 s


## <font color="red">Confrim if lat is below 50 and above -50
### <font color="green">We do this because the International Space Station does not travel past 50 and -50 degrees latitude.

### <font color="green">Verify function
- <font color="blue">Returns the min and max latitude in a tuple so the observer can see if the latitude matches the International Space Station path.

In [237]:
def verify(df: pd.DataFrame):
    return (df['lat'].min(), df['lat'].max())

In [238]:
verify(df)

(-51.79335497229585, 51.79347586838097)

In [240]:
df = df[['Date','lat','lon']]

In [251]:
df

,Date,lat,lon
0,2024-12-02 12:00:00,50.762893,24.239540
1,2024-12-02 12:04:00,51.429055,47.930295
2,2024-12-02 12:08:00,47.101542,69.774890
3,2024-12-02 12:12:00,39.058129,86.957239
4,2024-12-02 12:16:00,28.791850,100.005483
...,...,...,...
5397,2024-12-17 11:44:00,-51.361721,144.653539
5398,2024-12-17 11:48:00,-46.896149,166.347864
5399,2024-12-17 11:52:00,-38.765664,-176.623208
5400,2024-12-17 11:56:00,-28.444661,-163.673690


## <font color="red">Functions

### 1. <font color="green">future_date1
- <font color="blue">A function where you take the have argurements Pandas Dataframe and nhours.
The function takes the current time and adds in by nhours which the default is three. You will pass in a Pandas dataframe that will take the future values between startdate and 24 hours ahead.

### 2. <font color="green">future_dates
 - <font color="blue">A function where you take the have argurements Pandas Dataframe. The function takes the current time and adds in by three hours.


In [326]:
def future_date1(df: pd.DataFrame, nhours: int=3):
    # String of the current time of day now
    start_date = dt.datetime.now() + dt.timedelta(hours=nhours)
    #end_date = start_date + dt.timedelta(hours=24)
    # Turns the date from a string object to a datetime object
    start_date = start_date.strftime("%m/%d/%Y %H:%M:%S")
    start_date = pd.to_datetime(start_date, format='%m/%d/%Y %H:%M:%S')

    end_date = start_date + dt.timedelta(hours=24)
    #end_date = end_date.strftime("%m/%d/%Y %H:%M:%S")
    #end_date = pd.to_datetime(start_date, format='%m/%d/%Y %H:%M:%S')
    #return (start_date, end_date)

    return df[(start_date <= df['Date']) & (end_date >= df['Date'])]

In [328]:
df1 = future_date1(df)

In [329]:
df1

,Date,lat,lon
520,2024-12-03 22:40:00,-34.751532,-5.781202
521,2024-12-03 22:44:00,-43.923144,9.400623
522,2024-12-03 22:48:00,-50.081550,29.275880
523,2024-12-03 22:52:00,-51.699414,52.722728
524,2024-12-03 22:56:00,-48.235648,75.228904
...,...,...,...
875,2024-12-04 22:19:30,-22.264652,121.586177
876,2024-12-04 22:23:30,-10.385008,131.075651
877,2024-12-04 22:27:30,1.824014,139.789839
878,2024-12-04 22:31:30,13.979453,148.637757


In [213]:
def future_dates(df: pd.DataFrame):
    # String of the current time of day now
    start_date = dt.datetime.now() + dt.timedelta(hours=3)
    # Turns the date from a string object to a datetime object
    start_date = start_date.strftime("%m/%d/%Y %H:%M:%S")
    start_date = pd.to_datetime(start_date, format='%m/%d/%Y %H:%M:%S')
    #Check if value is date is in the future

    c = 0

    for element in df['Date']:
        c+=1
        if element > start_date:
            # Returning the date, lat, and lon
            df = df.iloc[c:]
            return df
        else:
            pass




In [214]:
df2 = future_dates(df)

In [ ]:
df2